In [42]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv, find_dotenv
from langchain import PromptTemplate
import os

# LLM Model and API Key

In [43]:
# Locate .env file, and load environment variables.
dotenv_path = find_dotenv()
if dotenv_path:
    load_dotenv(dotenv_path)
else:
    raise FileNotFoundError(".env file not found!")


# Set the model name for our LLMs.
GEMINI_MODEL = "gemini-2.0-flash"

# Load the API key to a variable.
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY is missing! Check your .env file.")


# Basic Query

In [44]:
# Initialize the model.
llm = ChatGoogleGenerativeAI(model=GEMINI_MODEL, google_api_key=GEMINI_API_KEY, temperature=0.3)

In [54]:
# restaurant_name = "Pho Binh"
# restaurant_city = "Houston"
# restaurant_street = "Beechnut"

restaurant_name = "Biryani Pot"
restaurant_city = "Houston"
restaurant_street = "Westheimer"

# restaurant_name = "Chama Gaucha Brazilian Steakhouse"
# restaurant_city = "Houston"
# restaurant_street = "Westheimer"

# Define a query as a string.
# query = f"Give me a summary and review of {restaurant_name} in {restaurant_city} on {restaurant_street}"

query = f"Give me the details of {restaurant_name} in {restaurant_city} on {restaurant_street}"

# Pass the query to the invoke method and print the result.
response = llm.invoke(query)
print(response.content)

Okay, here's the information I could find about Biryani Pot on Westheimer in Houston:

*   **Address:** 5655 Westheimer Rd Suite A, Houston, TX 77056
*   **Phone:** (713) 357-9777
*   **Cuisine:** Indian, Pakistani, Biryani
*   **Hours:**
    *   Sunday - Thursday: 11 AM - 10 PM
    *   Friday - Saturday: 11 AM - 11 PM
*   **Menu Highlights:** Known for their various biryanis, curries, and Indo-Chinese dishes.
*   **Service Options:**
    *   Dine-in
    *   Takeout
    *   Delivery
*   **Online Presence:** They have a website and are active on social media.
*   **Price:** $$ (Mid-range)


# Templates for Instructions

In [55]:
# Define the format for the template.

format = """You are a world famous restaurant critic.  Answer only questions that would be about restaurants.
If the human asks questions not related to restaurant, remind them that your job is to help them get the details of a restaurnt
Question: {query}
Answer:

    **Your response must be in the following format:**

    **Summary:** [Summary of the restaurant]
    **Moods:** [Moods of the restaurant]
    **Highlight:** [quality of food, service, value]
    **Rating:** [Rating]
    **Hours:** [hours of business]
    **Price:** [usual price range per person]

    For example:
    **Summary:** Chama Gaucha is a high-end churrascaria (Brazilian steakhouse) offering a wide selection of grilled meats carved tableside by gauchos (Brazilian cowboys)
    **Moods:** Vibrant, lively, and celebratory
    **Highlight:** Reviewers consistently praise the "perfectly cooked meats" and the "robust salad bar" with fresh options. 
                The service is frequently described as "impeccable" and "attentive," with staff ensuring glasses are filled and meat preferences are met.
    **Rating:** 4.7
    **Hours:**  *   Sunday - Thursday: 11 AM - 10 PM
                *   Friday - Saturday: 11 AM - 11 PM
    **Price:**  $30 - $50

    Do not include any extra text or formatting. 
"""

# Construct the prompt template.
prompt_template = PromptTemplate(
    input_variables=["query"], 
    template = format)

In [56]:
response = (prompt_template | llm).invoke(query)
print(response.content)  

**Summary:** Biryani Pot on Westheimer is a casual Indian restaurant specializing in various biryanis and other Indian dishes.
**Moods:** Casual, family-friendly, and bustling
**Highlight:** Reviewers often praise the flavorful and aromatic biryanis, generous portion sizes, and reasonable prices.
**Rating:** 4.0
**Hours:** * Sunday - Thursday: 11 AM - 10 PM
            * Friday - Saturday: 11 AM - 11 PM
**Price:** $15 - $25


<!-- # Combining Chains -->